In [ ]:
import pathlib
import textwrap
import requests
import json
from google.colab import userdata
import google.generativeai as genai
import time
import os
from IPython.display import display
from IPython.display import Markdown

# 替换为您的 GitHub 用户名和个人访问令牌
username = userdata.get('GITHUB_NAME')
token = userdata.get('GITHUB_TOKEN')

# GitHub API URL
url = f'https://api.github.com/users/{username}/starred'
headers = {"Authorization": f"token {token}"}

# 初始化存储列表
starred_repos = []

# 先测试一次效果
response = requests.get(url, headers=headers)
if response.status_code == 200:
    repos = response.json()
    starred_repos.extend(repos)
    # 输出调试信息
    print(f"第一次请求成功，获取到 {len(repos)} 个仓库信息")
    # 获取下一页的 URL（如果有）
    url = response.links.get('next', {}).get('url')
else:
    print(f"请求失败，状态码：{response.status_code}")
    url = None  # 停止循环

# 循环读取所有分页的仓库数据
while url:
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        repos = response.json()
        starred_repos.extend(repos)
        # 输出调试信息
        print(f"请求成功，获取到 {len(repos)} 个仓库信息，当前已获取 {len(starred_repos)} 个仓库")
        # 获取下一页的 URL（如果有）
        url = response.links.get('next', {}).get('url')
    else:
        print(f"请求失败，状态码：{response.status_code}")
        break

# 保存到本地 star.md 文件
with open('stars.md', 'w', encoding='utf-8') as file:
    for repo in starred_repos:
        name = repo['name']
        description = repo['description'] or 'No description'
        html_url = repo['html_url']
        category = 'Unknown Category'  # 默认分类
        tags = []  # 默认无标签
        # 格式化写入
        file.write(f"- **Name**: {name}\n")
        file.write(f"  - **Description**: {description}\n")
        file.write(f"  - **URL**: {html_url}\n")
        file.write(f"  - **Category**: {category}\n")
        file.write(f"  - **Tags**: {tags}\n\n")

html_url = []
for repo in starred_repos:
    html_url.append({"name": repo['name'], "url": repo['html_url']})  # 使用 append 添加元素

file_path = "./data.json"
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(html_url, f, indent=4, ensure_ascii=False)  # 将 html_url 写入文件
print(f"已保存仓库信息到 {file_path}")


In [ ]:
!apt-get install -y iputils-ping


In [ ]:
!ping google.com

处理json

In [ ]:
import pathlib
import textwrap
import requests
import json
from google.colab import userdata
import google.generativeai as genai
import time
import os
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
    text = text.replace("•", "  *")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))



gemini_api_key = userdata.get("GEMINI_API_KEY")

genai.configure(api_key=gemini_api_key)

# for m in genai.list_models():
#     if "generateContent" in m.supported_generation_methods:
#         print(m.name)

model = genai.GenerativeModel("gemini-1.5-flash")

def generate_repo_info(repo_url):
    """使用 Gemini 生成 JSON 格式的描述、分类和标签。

    Args:
        repo_url: 仓库 URL。

    Returns:
        一个包含描述、分类和标签的字典，如果生成失败则返回 None。
    """
    prompt = f"""请根据以下 GitHub 仓库链接生成 JSON 格式的描述、分类和标签：
{repo_url}

JSON 格式：
{{
  "description": "仓库描述",
  "category": "仓库分类",
  "tags": ["标签1", "标签2", ...]
}}
"""
    response = model.generate_content(prompt)
    try:
        # 使用 strip() 去除空白字符
        # print(f"Gemini API 返回的 JSON 数据：{response.text}")
        res_text=response.text.replace("```json", "")
        res_text=res_text.replace("```", "")
        # print(f"JSON 数据：{res_text}")
        repo_info = json.loads(res_text.strip())
        return repo_info
    except json.JSONDecodeError:
        print(f"无法解析 Gemini 返回的 JSON 数据：{response.text}")
        return None



file_path="./data.json"
# 读取 stars.md 文件并更新信息

with open(file_path, 'r', encoding='utf-8') as f:
    jsonData = json.load(f)


updated_lines = []
processed_count = 0  # 记录已处理的仓库数量

# json_dir = './json'
# for filename in os.listdir(json_dir):
#     json_path = os.path.join(json_dir, filename)
#     if os.path.isfile(json_path):  # 确保它是一个文件而不是目录
#         os.remove(json_path)
#         print(f"已删除文件: {json_path}")

# 循环处理所有仓库
for item in jsonData:
    name = item['name']
    url = item['url']
    # 使用 Gemini 生成仓库信息
    print(f"Name: {name}, URL: {url}")
    onefile = f"./json/{name}.json"
    if os.path.isfile(onefile):
      print(f"文件已存在，跳过：{onefile}")
      continue
    else:
      print(f"获取git conten：{onefile}")
      repo_info = generate_repo_info(item['url'])
      print(f"repo_info {repo_info}")

      if repo_info:

        with open(onefile, 'w', encoding='utf-8') as f:
          json.dump(html_url, f, indent=4, ensure_ascii=False)
        print(f"已保存仓库信息到 {onefile}")
        # break
        # 更新描述、分类和标签
        updated_lines.append(f"- **Name**: {name}\n")
        updated_lines.append(f"  - **Description**: {repo_info['description']}\n")
        updated_lines.append(f"  - **URL**: {url}\n")
        updated_lines.append(f"  - **Category**: {repo_info['category']}\n")
        updated_lines.append(f"  - **Tags**: {repo_info['tags']}\n\n")
        processed_count += 1
        print(f"已处理 {processed_count} 个仓库，当前仓库：{name}")
        time.sleep(2)
        # break
      else:
        # 如果 Gemini 生成信息失败，保留原始行
        updated_lines.append(f"- **Name**: {name}\n")
        updated_lines.append(f"  - **URL**: {url}\n")

        print(f"Gemini 生成信息失败，跳过仓库：{name}")

    # break  # 仅处理第一个仓库，用于测试

# 将更新后的内容写回 stars.md 文件
with open('./new_stars.md', 'w', encoding='utf-8') as file:
    file.writelines(updated_lines)

print('new_stars.md 文件更新成功！')


